# Multiple Object Detection and Tracking for waste segregation

## Installation

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -r /content/drive/MyDrive/inzynierka/project_repo /content/

In [ ]:
%cd /content/project_repo
!pip install filterpy
# !pip install -r /content/project/requierements.txt
%cd /content

/content/project
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 KB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110474 sha256=157c0bd7fa973c1c3a1682e0141c544aa3414ba937218193f3f5bc4093c3851e
  Stored in directory: /root/.cache/pip/wheels/fe/f6/cb/40331472edf4fd399b8cad02973c6acbdf26898342928327fe
Successfully built filterpy
/content


In [ ]:
import os, sys, time, datetime, random
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.autograd import Variable

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

In [ ]:
!python -m pip install pyyaml==5.1
# Detectron2 has not released pre-built binaries for the latest pytorch (https://github.com/facebookresearch/detectron2/issues/4053)
# so we install from source instead. This takes a few minutes.
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

# Install pre-built detectron2 that matches pytorch version, if released:
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
#!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/{CUDA_VERSION}/{TORCH_VERSION}/index.html

# exit(0)  # After installation, you may need to "restart runtime" in Colab. This line can also restart runtime

import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.2/274.2 KB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyyaml: filename=PyYAML-5.1-cp38-cp38-linux_x86_64.whl size=44089 sha256=9c54522eb94fae6299b8cd81cd23b380a44f2c8a6161fe4bea2c43c8bafdbb57
  Stored in directory: /root/.cache/pip/wheels/52/dd/2b/10ff8b0ac81b93946bb5fb9e6749bae2dac246506c8774e6cf
Successfully built pyyaml
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0
    Uninstalling PyYAML-6.0:
      Successfully uninstalled PyYAML-6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dask 2022.2.1 requires pyyaml>=5.3.1, but you have pyyaml 5.1 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheel

In [ ]:
# Some basic setup:
# Setup detectron2 logger
import torch, detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow
# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.utils.visualizer import ColorMode
import random

## Dataset Registration

In [ ]:
# DatasetCatalog.clear()
from detectron2.data.datasets import register_coco_instances
register_coco_instances("zerowaste_test", {}, "/content/drive/MyDrive/inzynierka/datasets/zerowaste-f-final/splits_final_deblurred/test/labels.json", "/content/drive/MyDrive/inzynierka/datasets/zerowaste-f-final/splits_final_deblurred/test/data")
zerowaste_test_metadata = MetadataCatalog.get("Zerowaste_test")
dataset_dicts_test = DatasetCatalog.get("zerowaste_test")

[01/20 21:17:58 d2.data.datasets.coco]: Loading /content/drive/MyDrive/inzynierka/datasets/zerowaste-f-final/splits_final_deblurred/test/labels.json takes 1.55 seconds.
[01/20 21:17:58 d2.data.datasets.coco]: Loaded 929 images in COCO format from /content/drive/MyDrive/inzynierka/datasets/zerowaste-f-final/splits_final_deblurred/test/labels.json


In [ ]:
from detectron2.config import get_cfg
import os

cfg = get_cfg()
cfg.merge_from_file(
    "/content/drive/MyDrive/inzynierka/project/config/detectron2_custom_congif_zerowaste_maskrcnn_R_101_FPN_3x.yaml"
)
predictor = DefaultPredictor(cfg)

[01/20 21:18:04 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /content/drive/MyDrive/inzynierka/project/detection/maskrcnn/output_zerowasteaug/model_final.pth ...


# SORT

In [ ]:
%%writefile /content/project/a.py
import torch, detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
# import some common libraries
import numpy as np
import cv2
from google.colab.patches import cv2_imshow
# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.video_visualizer import VideoVisualizer
from detectron2.utils.visualizer import ColorMode, Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.autograd import Variable

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

import os, sys, time, datetime, random, json
from time import sleep


import multiprocessing.queues as mpq
import multiprocessing as mp
import queue

from detectron2.config import get_cfg
import os
from detectron2.data.datasets import register_coco_instances

from utils.utils import *
from utils.FrameQueue import FrameQueue, MyManager

from tracking.sort.sort import *


def detector(frameQueue, detections_queue, allow_tracker,end_detector, predictor,detector_ready,end_tracker,img_scale_percent):
    print('detector:', mp.current_process())


    
    width = int(1920 * img_scale_percent / 100)
    height = int(1080 * img_scale_percent / 100)

    print('detector dummy frame:', [height,width,3])

    dummy_frame = np.zeros([height,width,3],dtype=np.uint8)
    # predictor(np.zeros([height,width,3],dtype=np.uint8))
    start_dummy = time.time()
    detections_output, model_output = detect_image_maskrcnn(dummy_frame, predictor)
    print('dummy pred time: ', time.time() - start_dummy)

    print('predicted dummy')
    detector_ready.set()
    
    while True:
        
        if frameQueue.qsize() > 0:
            break

    while True:
        detector_start = time.time()
        print('Detector frameQueue:')
        try:
            frame, frame_num = frameQueue.get_detection_frame()
        except queue.Empty:
            print('empty')
            break

        print('Detector got frame: ', time.time()-detector_start)
        print(f"Detector: Running detection on frame {frame_num}")
        ###################
        det_start = time.time()
        detections_output, model_output = detect_image_maskrcnn(frame, predictor)
        print('det time:', time.time()-det_start)
        # DETECTION PROCESS
        ###################
        print(f"Detector: Done frame {frame_num}")
        if allow_tracker.is_set()==False:
            allow_tracker.set()
        detections_queue.put((frame, frame_num, detections_output, model_output))
        detector_stop = time.time()
        print('detector loop time:', detector_stop - detector_start)
    end_detector.set()

    while not end_tracker.is_set():
        time.sleep(1)
        
    print(' === Detector off === ')

def tracker(frameQueue, detections_queue, result_queue,allow_tracker,end_tracker,fps,img_scale_percent):
    mot_tracker = Sort(max_age=6, min_hits=0, iou_threshold=0.2, img_scale_percent=img_scale_percent) 
    sampling_time = 1/fps
    print('tracker:', mp.current_process())
    detection = None
    while True:
        tracker_start = time.time()

        if not allow_tracker.is_set():
            #print('tracker waiting...', frameQueue.qsize())
            #time.sleep(0.5)
            continue
        
        try:
            tracking_frame = frameQueue.get_tracking_frame()
        except queue.Empty:
            print('empty')
            break

        if detection != None:
            while detection[1] < tracking_frame[1] and detections_queue.qsize()>0:
                detection = detections_queue.get()

        detections_queue.qsize()
        if detections_queue.qsize()>0:
            if detection==None:
                detection = detections_queue.get()
                if detection[1] == tracking_frame[1]:
                    print(f"Tracker: Running tracking WITH detections {detection[1]} on frame {tracking_frame[1]}")
                    ##################
                    tracked_objects = mot_tracker.update(detection[2])
                    ##################
                    print(f"Tracker: Done frame {tracking_frame[1]}")
                    result_queue.put((tracking_frame, tracked_objects, detection))
                    detection = None
                else:
                    print(f"Tracker: Running tracking WITHOUT detections on frame {tracking_frame[1]}")

                    ##################
                    tracked_objects = mot_tracker.update(dets=np.empty((0, 5)))
                    ##################
                    print(f"Tracker: Done frame {tracking_frame[1]}")
                    result_queue.put((tracking_frame, tracked_objects, None))

            else: #  detection!=None
                if detection[1] == tracking_frame[1]:
                    print(f"Tracker: Running tracking WITH detections {detection[1]} on frame {tracking_frame[1]}")
                    ##################
                    tracked_objects = mot_tracker.update(detection[2])
                    ##################
                    print(f"Tracker: Done frame {tracking_frame[1]}")
                    result_queue.put((tracking_frame, tracked_objects, detection))
                    detection = None

                else:
                    print(f"Tracker: Running tracking WITHOUT detections on frame {tracking_frame[1]}")
                    ##################
                    tracked_objects = mot_tracker.update(dets=np.empty((0, 5)))
                    ##################
                    print(f"Tracker: Done frame {tracking_frame[1]}")
                    result_queue.put((tracking_frame, tracked_objects, None))
        else: # detections_queue.qsize() == 0
            if detection != None:
                if detection[1] == tracking_frame[1]:
                    print(f"Tracker: Running tracking WITH detections {detection[1]} on frame {tracking_frame[1]}")
                    ##################
                    tracked_objects = mot_tracker.update(detection[2])
                    ##################
                    print(f"Tracker: Done frame {tracking_frame[1]}")
                    result_queue.put((tracking_frame, tracked_objects, detection))
                    # result_queue.task_done()
                    detection = None
                else:
                    print(f"Tracker: Running tracking WITHOUT detections on frame {tracking_frame[1]}")
                    ##################
                    tracked_objects = mot_tracker.update(dets=np.empty((0, 5)))
                    ##################
                    print(f"Tracker: Done frame {tracking_frame[1]}")
                    result_queue.put((tracking_frame, tracked_objects, None))
            else:
                print(f"Tracker: Running tracking WITHOUT detections on frame {tracking_frame[1]}")
                ##################
                tracked_objects = mot_tracker.update(dets=np.empty((0, 5)))
                ##################
                print(f"Tracker: Done frame {tracking_frame[1]}")
                result_queue.put((tracking_frame, tracked_objects, None))

        diff = time.time() - tracker_start
        while  diff < sampling_time and frameQueue.qsize()<=3:
            diff = time.time() - tracker_start
        print('tracker time:', diff)


    # cleaning detections
    end_tracker.set()
    while detections_queue.qsize()>0:
        detections_queue.get()

    while result_queue.qsize()>0:
        time.sleep(1)
        continue

    print(' === Tracker off === ')



def system(videopath, frameQueue, detections_queue, result_queue, allow_tracker, end_detector, end_tracker,dataset_metadata_catalog, detector_ready,fps,img_scale_percent):

    print('system:', mp.current_process())
    ########### Video ###################
    video = cv2.VideoCapture(videopath)
    if (video.isOpened() == False): 
        print("Error reading video file")
    #fps = video.get(cv2.CAP_PROP_FPS)
    sampling_time = 1/fps
    print('sampling_time:',sampling_time)
    frame_width = int(video.get(3))
    frame_height = int(video.get(4))
    size = (frame_width, frame_height)

    #scale_percent = 50 # percent of original size
    width = int(frame_width * img_scale_percent / 100)
    height = int(frame_height * img_scale_percent / 100)
    dim = (width, height)
    ###################################
    result = cv2.VideoWriter(f"testing_sort-{img_scale_percent}-{fps}fps.avi", 
                            cv2.VideoWriter_fourcc(*'MJPG'),
                            fps, dim)
    ###################################
    class_path='config/zerowaste.names'
    classes = load_classes(class_path)
    counter = 1
    system_start = time.time()
    while(True):
        if not detector_ready.is_set():
            continue

        start = time.time()

        ret, frame = video.read()
        if ret == True:
            print('------------------ ' + str(counter) +' ------------------')
            resized_frame = cv2.resize(frame, dim, interpolation = cv2.INTER_AREA)
            print('resized frame shape', resized_frame.shape)
            frameQueue.add_frame((resized_frame, counter))
            print(time.time())
            # print('frameQueue:', [element[1] for element in frameQueue.queue])
            frameQueue.print()
            

            if allow_tracker.is_set() == False:      
                print('--> Add to queue and go to next frame')
                counter += 1
                diff = time.time() - start

                while  diff < sampling_time:
                    diff = time.time() - start
                print(diff) 
                continue

        elif ret == False:
            break
        
        counter += 1

        diff = time.time() - start
        while  diff < sampling_time:
            diff = time.time() - start
        print(diff)
        

    while True:
        if end_detector.is_set() and end_tracker.is_set():
            break
    system_stop = time.time()
    print('System time = ', system_stop-system_start)

    v = VideoVisualizer(dataset_metadata_catalog, ColorMode.IMAGE)
    print(' \\\\\\\\\\\ Saving frames to video')
    while result_queue.qsize()>0: # and (detections_queue.empty() and tracking_queue.empty())
        (tracking_frame, tracking_frame_num), tracked_objects, detection = result_queue.get()
        if detection is not None:
            detection_frame, detection_frame_num, detections_output, model_output = detection
            visualizer = v.draw_instance_predictions(detection_frame, model_output["instances"])
            img_pred = visualizer.get_image()
            cv2.putText(img_pred,  "DETECTION", (5, 50), cv2.FONT_HERSHEY_DUPLEX, 0.8, (0,255,0), 2)
            tracking_frame = img_pred
        for x1, y1, x2, y2, obj_id, cls_pred in tracked_objects:
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            cls = classes[int(cls_pred)]
            cv2.rectangle(tracking_frame, (x1, y1), (x2, y2), (0, 0, 255), 3)
            cv2.putText(tracking_frame, cls + "-" + str(int(obj_id)), (x1, y1 + 20), cv2.FONT_HERSHEY_DUPLEX, 0.8, (255,255,255), 2)

        cv2.putText(tracking_frame,  f"FRAME: {tracking_frame_num} | FPS: {fps}", (5, 25), cv2.FONT_HERSHEY_DUPLEX, 1, (255,255,255), 2)
        result.write(tracking_frame)
    ########################################

    result.release()
    video.release()
    # Closes all the frames
    cv2.destroyAllWindows()
    print("The video was successfully saved")


def detect_and_track_zerowaste(predictor,dataset_metadata_catalog):

    MyManager.register('FrameQueue', FrameQueue)

    fps = 30
    img_scale_percent = 100
    
    # create a new manager instance
    with MyManager() as manager:
        # create a shared custom class instance
        frameQueue = manager.FrameQueue()
        detections_queue = mp.Queue()#mp.Queue()
        result_queue = mp.Queue()#mp.Queue()
        videopath = '/content/drive/MyDrive/inzynierka/project/videos/05_frame-10-12.mp4'

        allow_tracker = mp.Event()
        end_detector = mp.Event()
        end_tracker = mp.Event()
        detector_ready = mp.Event()

        tracker_p = mp.Process(target=tracker, args=(frameQueue,detections_queue,result_queue,allow_tracker,end_tracker,fps,img_scale_percent,))
        tracker_p.start()

        detector_p = mp.Process(target=detector, args=(frameQueue,detections_queue,allow_tracker,end_detector,predictor,detector_ready,end_tracker,img_scale_percent))
        detector_p.start()

        
        system_p = mp.Process(target=system, args=(videopath, frameQueue, detections_queue, result_queue, allow_tracker, end_detector, end_tracker,dataset_metadata_catalog,detector_ready,fps,img_scale_percent,))
        system_p.start()
        #system(videopath, frameQueue, detections_queue, result_queue, allow_tracker, end_detector, end_tracker,dataset_metadata_catalog,detector_ready,fps)


        detector_p.join()
        tracker_p.join()
        system_p.join()


        #system(videopath, frameQueue, detections_queue, result_queue, allow_tracker, end_detector, end_tracker )



if __name__=="__main__":

    mp.set_start_method("spawn")
    register_coco_instances("zerowaste_test", {}, "/content/drive/MyDrive/inzynierka/datasets/zerowaste-f-final/splits_final_deblurred/test/labels.json", "/content/drive/MyDrive/inzynierka/datasets/zerowaste-f-final/splits_final_deblurred/test/data")
    zerowaste_test_metadata = MetadataCatalog.get("zerowaste_test")
    dataset_dicts_test = DatasetCatalog.get("zerowaste_test")



    cfg = get_cfg()
    cfg.merge_from_file(
        "/content/drive/MyDrive/inzynierka/project/config/detectron2_custom_congif_zerowaste_maskrcnn_R_101_FPN_3x.yaml"
    )
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.65   # set the testing threshold for this model
    predictor = DefaultPredictor(cfg)

    
    detect_and_track_zerowaste(predictor=predictor, dataset_metadata_catalog=zerowaste_test_metadata)

Overwriting /content/project/a.py


In [ ]:
!python a.py

[01/23 00:56:41 d2.data.datasets.coco]: Loaded 929 images in COCO format from /content/drive/MyDrive/inzynierka/datasets/zerowaste-f-final/splits_final_deblurred/test/labels.json
[01/23 00:56:45 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /content/drive/MyDrive/inzynierka/project/detection/maskrcnn/output_zerowasteaug/model_final.pth ...
tracker: <Process name='Process-2' parent=6329 started>
detector: <Process name='Process-3' parent=6329 started>
detector dummy frame: [1080, 1920, 3]
system: <Process name='Process-4' parent=6329 started>
sampling_time: 0.03333333333333333
/usr/local/lib/python3.8/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
dummy pred time:  4.056997537612915
predicted dummy
------------------ 1 -----

In [ ]:
!python a.py

[01/21 14:58:22 d2.data.datasets.coco]: Loaded 929 images in COCO format from /content/drive/MyDrive/inzynierka/datasets/zerowaste-f-final/splits_final_deblurred/test/labels.json
[01/21 14:58:26 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /content/drive/MyDrive/inzynierka/project/detection/maskrcnn/output_zerowasteaug/model_final.pth ...
tracker: <Process name='Process-2' parent=45216 started>
detector: <Process name='Process-3' parent=45216 started>
detector dummy frame: [1080, 1920, 3]
system: <Process name='Process-4' parent=45216 started>
sampling_time: 0.08333333333333333
/usr/local/lib/python3.8/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
dummy pred time:  4.147636651992798
predicted dummy
------------------ 1 --

# DeepSORT

In [ ]:
!cd /content/project/tracking/deepsort/deep_sort_realtime-master && pip3 install .
%cd /content/project

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/project/tracking/deepsort/deep_sort_realtime-master
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for deep-sort-realtime: filename=deep_sort_realtime-1.3.1-py3-none-any.whl size=8429591 sha256=efc8c360086b2b3dcc4adffea4d7d5bad1ed1700385338819da8f8de962a9381
  Stored in directory: /root/.cache/pip/wheels/2e/7a/55/ec2db3b06bea5ee3bf47382a9d4c33ec12d26df98a1b77cf18
Successfully built deep-sort-realtime
/content/project


DODAC ZDJECIE `/content/project/01_frame_000680 (1).PNG`

In [ ]:
%%writefile /content/project/b.py
import torch, detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
# import some common libraries
import numpy as np
import cv2
from google.colab.patches import cv2_imshow
# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.video_visualizer import VideoVisualizer
from detectron2.utils.visualizer import ColorMode, Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.autograd import Variable

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

import os, sys, time, datetime, random, json
from time import sleep

from multiprocessing.managers import BaseManager
import multiprocessing.queues as mpq
import multiprocessing as mp
import queue

from detectron2.config import get_cfg
import os
from detectron2.data.datasets import register_coco_instances

from utils.utils import *
from utils.FrameQueue import FrameQueue, MyManager


# DeepSORT
from deep_sort_realtime.deepsort_tracker import DeepSort
from deep_sort_realtime.deep_sort import kalman_filter


def convert_detections_for_deepsort(detections):
    result = []
    for detection in detections:
        x1, y1, x2, y2 = detection[:4]
        x1, x2, w, h = x1y1x2y2_to_x1y1wh(x1.detach().numpy(), y1.detach().numpy(), x2.detach().numpy(), y2.detach().numpy())
        result.append(([x1, x2, w, h], detection[4].detach().numpy(), detection[5].detach().numpy()))
    return result


def detector(frameQueue, detections_queue, allow_tracker,end_detector, predictor,detector_ready,end_tracker,img_scale_percent):
    print('detector:', mp.current_process())

    width = int(1920 * img_scale_percent / 100)
    height = int(1080 * img_scale_percent / 100)

    print('detector dummy frame:', [height,width,3])

    dummy_frame = np.zeros([height,width,3],dtype=np.uint8)
    # predictor(np.zeros([height,width,3],dtype=np.uint8))
    start_dummy = time.time()
    detections_output, model_output = detect_image_maskrcnn(dummy_frame, predictor)
    print('dummy pred time: ', time.time() - start_dummy)

    print('predicted dummy')
    detector_ready.set()
    
    while True:
        
        if frameQueue.qsize() > 0:
            break

    while True:
        detector_start = time.process_time()
        # if frameQueue.qsize() > 0:
        # frame, frame_num = frameQueue.get_detection_frame()
        try:
            frame, frame_num = frameQueue.get_detection_frame()
            
            # frameQueue.print_det_size()
        except queue.Empty:
            print('empty')
            break

        print(f"Detector: Running detection on frame {frame_num}")
        ###################
        det_start = time.process_time()
        detections_output, model_output = detect_image_maskrcnn(frame, predictor)
        print('det time:', time.process_time()-det_start)
        # DETECTION PROCESS
        ###################
        print(f"Detector: Done frame {frame_num}")
        if allow_tracker.is_set()==False:
            allow_tracker.set()
        detections_queue.put((frame, frame_num, detections_output, model_output))
        detector_stop = time.process_time()
        print('detector loop time:', detector_stop - detector_start)
    end_detector.set()

    while not end_tracker.is_set():
        time.sleep(1)
        continue
    print(' === Detector off === ')

def tracker(frameQueue, detections_queue, result_queue,allow_tracker,end_tracker,mot_tracker,fps,img_scale_percent,tracker_ready):

    print('tracker:', mp.current_process())

    sampling_time = 1/fps
    dummy_detections =torch.tensor([[258.7459, 304.2128, 342.2893, 343.4235,   0.8144,   1.0000],
                                    [405.6260, 137.3414, 519.5634, 293.4001,   0.7609,   3.0000],
                                    [546.6186,  79.7919, 657.5923, 165.6807,   0.7551,   1.0000],
                                    [436.6346,  74.1342, 547.4369, 163.3923,   0.7205,   1.0000]])
    dets = convert_detections_for_deepsort(dummy_detections)
    width = int(1920 * img_scale_percent / 100)
    height = int(1080 * img_scale_percent / 100)
    dim = (width, height)

    print('TRACKER dummy frame:', [height,width,3])
    #dummy_frame = cv2.imread("/content/project/01_frame_000680 (1).PNG")
    dummy_frame = np.zeros([height,width,3],dtype=np.uint8)
    dummy_frame = cv2.resize(dummy_frame, dim, interpolation = cv2.INTER_AREA)

    start_dummy = time.time()
    mot_tracker.update_tracks(dets, frame=dummy_frame)
    print('TRACKER dummy frame time: ', time.time() - start_dummy)
    mot_tracker.tracker.tracks = []

    tracker_ready.set()
    detection = None
    while True:
        
        tracker_start = time.time()

        if not allow_tracker.is_set():
            #print('=== tracker waiting ====')
            #time.sleep(0.5)
            continue

        try:

            tracking_frame = frameQueue.get_tracking_frame()
        except queue.Empty:
            print('empty')
            break

        if detection != None:
            while detection[1] < tracking_frame[1] and detections_queue.qsize()>0:
                detection = detections_queue.get()

        detections_queue.qsize()
        if detections_queue.qsize()>0:
            if detection==None:
                detection = detections_queue.get()
                if detection[1] == tracking_frame[1]:
                    print(f"Tracker: Running tracking WITH detections {detection[1]} on frame {tracking_frame[1]}")
                    ##################
                    print("OPTION 1")
                    dets = convert_detections_for_deepsort(detection[2])
                    tracks = mot_tracker.update_tracks(dets, frame=detection[0])
                    tracked_objects = []
                    for track_obj in tracks:
                        (x1, y1, x2, y2), obj_id, cls_pred = track_obj.to_ltrb(), track_obj.track_id, track_obj.get_det_class()
                        tracked_objects.append(((x1, y1, x2, y2), obj_id, cls_pred))
                    ##################
                    print(f"Tracker: Done frame {tracking_frame[1]}")
                    result_queue.put((tracking_frame, tracked_objects, detection))
                    detection = None
                else:
                    print(f"Tracker: Running tracking WITHOUT detections on frame {tracking_frame[1]}")
                    ##################
                    print("OPTION 2")
                    tracks = mot_tracker.update_tracks([], frame=tracking_frame[0])
                    tracked_objects = []
                    for track_obj in tracks:
                        (x1, y1, x2, y2), obj_id, cls_pred = track_obj.to_ltrb(), track_obj.track_id, track_obj.get_det_class()
                        tracked_objects.append(((x1, y1, x2, y2), obj_id, cls_pred))
                    ##################
                    print(f"Tracker: Done frame {tracking_frame[1]}")
                    result_queue.put((tracking_frame, tracked_objects, None))

            else: #  detection!=None
                if detection[1] == tracking_frame[1]:
                    print(f"Tracker: Running tracking WITH detections {detection[1]} on frame {tracking_frame[1]}")
                    ##################
                    print("OPTION 3")
                    dets = convert_detections_for_deepsort(detection[2])
                    tracks = mot_tracker.update_tracks(dets, frame=detection[0])
                    tracked_objects = []
                    for track_obj in tracks:
                        (x1, y1, x2, y2), obj_id, cls_pred = track_obj.to_ltrb(), track_obj.track_id, track_obj.get_det_class()
                        tracked_objects.append(((x1, y1, x2, y2), obj_id, cls_pred))
                    ##################
                    print(f"Tracker: Done frame {tracking_frame[1]}")
                    result_queue.put((tracking_frame, tracked_objects, detection))
                    detection = None

                else:
                    print(f"Tracker: Running tracking WITHOUT detections on frame {tracking_frame[1]}")
                    ##################
                    print("OPTION 4")
                    tracks = mot_tracker.update_tracks([], frame=tracking_frame[0])
                    tracked_objects = []
                    for track_obj in tracks:
                        (x1, y1, x2, y2), obj_id, cls_pred = track_obj.to_ltrb(), track_obj.track_id, track_obj.get_det_class()
                        tracked_objects.append(((x1, y1, x2, y2), obj_id, cls_pred))
                    ##################
                    print(f"Tracker: Done frame {tracking_frame[1]}")
                    result_queue.put((tracking_frame, tracked_objects, None))
        else: # detections_queue.qsize() == 0
            if detection != None:
                if detection[1] == tracking_frame[1]:
                    print(f"Tracker: Running tracking WITH detections {detection[1]} on frame {tracking_frame[1]}")
                    ##################
                    print("OPTION 5")
                    dets = convert_detections_for_deepsort(detection[2])
                    tracks = mot_tracker.update_tracks(dets, frame=detection[0])
                    tracked_objects = []
                    for track_obj in tracks:
                        (x1, y1, x2, y2), obj_id, cls_pred = track_obj.to_ltrb(), track_obj.track_id, track_obj.get_det_class()
                        tracked_objects.append(((x1, y1, x2, y2), obj_id, cls_pred))
                    ##################
                    print(f"Tracker: Done frame {tracking_frame[1]}")
                    result_queue.put((tracking_frame, tracked_objects, detection))
                    # result_queue.task_done()
                    detection = None
                else:
                    print(f"Tracker: Running tracking WITHOUT detections on frame {tracking_frame[1]}")
                    ##################
                    print("OPTION 6")
                    tracks = mot_tracker.update_tracks([], frame=tracking_frame[0])
                    tracked_objects = []
                    for track_obj in tracks:
                        (x1, y1, x2, y2), obj_id, cls_pred = track_obj.to_ltrb(), track_obj.track_id, track_obj.get_det_class()
                        tracked_objects.append(((x1, y1, x2, y2), obj_id, cls_pred))
                    ##################
                    print(f"Tracker: Done frame {tracking_frame[1]}")
                    result_queue.put((tracking_frame, tracked_objects, None))
            else:
                print(f"Tracker: Running tracking WITHOUT detections on frame {tracking_frame[1]}")
                ##################
                print("OPTION 7")
                tracks = mot_tracker.update_tracks([], frame=tracking_frame[0])
                tracked_objects = []
                for track_obj in tracks:
                    (x1, y1, x2, y2), obj_id, cls_pred = track_obj.to_ltrb(), track_obj.track_id, track_obj.get_det_class()
                    tracked_objects.append(((x1, y1, x2, y2), obj_id, cls_pred))
                ##################
                print(f"Tracker: Done frame {tracking_frame[1]}")
                result_queue.put((tracking_frame, tracked_objects, None))

        diff = time.time() - tracker_start
        while  diff < sampling_time and frameQueue.qsize()<=3:
            diff = time.time() - tracker_start
        print('tracker time:', diff)


    # cleaning detections
    end_tracker.set()
    while detections_queue.qsize()>0:
        detections_queue.get()

    while result_queue.qsize()>0:
        time.sleep(1)
        continue

    print(' === Tracker off === ')



def system(videopath, frameQueue, detections_queue, result_queue, allow_tracker, end_detector, end_tracker,dataset_metadata_catalog, detector_ready,fps,img_scale_percent,tracker_ready):

    print('system:', mp.current_process())
    ########### Video ###################
    video = cv2.VideoCapture(videopath)
    if (video.isOpened() == False): 
        print("Error reading video file")
    #fps = video.get(cv2.CAP_PROP_FPS)
    sampling_time = 1/fps
    print('sampling_time:',sampling_time)
    frame_width = int(video.get(3))
    frame_height = int(video.get(4))
    size = (frame_width, frame_height)

    #scale_percent = 100 # percent of original size
    width = int(frame_width * img_scale_percent / 100)
    height = int(frame_height * img_scale_percent / 100)
    dim = (width, height)
    ###################################
    result = cv2.VideoWriter(f"deepsort-{img_scale_percent}-{fps}fps-pred_thres_065.avi", 
                            cv2.VideoWriter_fourcc(*'MJPG'),
                            fps, dim)
    ###################################
    class_path='config/zerowaste.names'
    classes = load_classes(class_path)
    counter = 1



    system_start = time.time()
    while(True):
        if not detector_ready.is_set():
            continue

        if not tracker_ready.is_set():
            continue

        start = time.time()

        ret, frame = video.read()
        if ret == True:
            print('------------------ ' + str(counter) +' ------------------')
            resized_frame = cv2.resize(frame, dim, interpolation = cv2.INTER_AREA)
            frameQueue.add_frame((resized_frame, counter))
            # print('frameQueue:', [element[1] for element in frameQueue.queue])
            frameQueue.print()

            if allow_tracker.is_set() == False:      
                print('--> Add to queue and go to next frame')
                counter += 1
                diff = time.time() - start

                while  diff < sampling_time:
                    diff = time.time() - start
                print(diff) 
                continue

        elif ret == False:
            break
        
        counter += 1

        diff = time.time() - start
        while  diff < sampling_time:
            diff = time.time() - start
        print(diff)
        

    while True:
        if end_detector.is_set() and end_tracker.is_set():
            break
    system_stop = time.time()
    print('System time = ', system_stop-system_start)
    
    v = VideoVisualizer(dataset_metadata_catalog, ColorMode.IMAGE)
    print(' \\\\\\\\\\\ Saving frames to video')
    while result_queue.qsize()>0: # and (detections_queue.empty() and tracking_queue.empty())
        (tracking_frame, tracking_frame_num), tracked_objects, detection = result_queue.get()
        if detection is not None:
            detection_frame, detection_frame_num, detections_output, model_output = detection
            visualizer = v.draw_instance_predictions(detection_frame, model_output["instances"])
            img_pred = visualizer.get_image()
            cv2.putText(img_pred,  "DETECTION", (5, 50), cv2.FONT_HERSHEY_DUPLEX, 0.8, (0,255,0), 2)
            tracking_frame = img_pred

        for (x1, y1, x2, y2), obj_id, cls_pred in tracked_objects:
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)     
            cls = classes[int(cls_pred)]
            cv2.rectangle(tracking_frame, (x1, y1), (x2, y2), (0, 0, 255), 3)
            cv2.putText(tracking_frame, cls + "-" + str(int(obj_id)), (x1, y2 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 3)

        cv2.putText(tracking_frame,  f"FRAME: {tracking_frame_num} | FPS: {fps}", (5, 25), cv2.FONT_HERSHEY_DUPLEX, 1, (255,255,255), 2)
        result.write(tracking_frame)
    ########################################

    result.release()
    video.release()
    # Closes all the frames
    cv2.destroyAllWindows()
    print("The video was successfully saved")


def detect_and_track_zerowaste(predictor,dataset_metadata_catalog):

    MyManager.register('FrameQueue', FrameQueue)

    fps =25
    img_scale_percent = 100


    # create a new manager instance
    with MyManager() as manager:
        # create a shared custom class instance
        frameQueue = manager.FrameQueue()
        detections_queue = mp.Queue()#mp.Queue()
        result_queue = mp.Queue()#mp.Queue()
        videopath = '/content/drive/MyDrive/inzynierka/project/videos/05_frame-10-12.mp4'
        #videopath = '/content/project/videos/08_frame-10-12.mp4'

        allow_tracker = mp.Event()
        end_detector = mp.Event()
        end_tracker = mp.Event()
        detector_ready = mp.Event()
        tracker_ready = mp.Event()

        detector_p = mp.Process(target=detector, args=(frameQueue,detections_queue,allow_tracker,end_detector,predictor,detector_ready,end_tracker,img_scale_percent,))
        detector_p.start()
        #detector_p = torch.multiprocessing.spawn(detector, args=(frameQueue,detections_queue,allow_tracker,end_detector,predictor,detector_ready,end_tracker,img_scale_percent,), nprocs=1, join=False, daemon=False, start_method='spawn')

        tracker_p = mp.Process(target=tracker, args=(frameQueue,detections_queue,result_queue,allow_tracker,end_tracker,fps,img_scale_percent,tracker_ready,))
        tracker_p.start()
        #tracker_p = torch.multiprocessing.spawn(tracker, args=(frameQueue,detections_queue,result_queue,allow_tracker,end_tracker,fps,img_scale_percent,tracker_ready,), nprocs=1, join=False, daemon=False, start_method='spawn')

        system_p = mp.Process(target=system, args=(videopath, frameQueue, detections_queue, result_queue, allow_tracker, end_detector, end_tracker,dataset_metadata_catalog,detector_ready,fps,img_scale_percent,tracker_ready,))
        system_p.start()
        #system_p = torch.multiprocessing.spawn(system, args=(videopath, frameQueue, detections_queue, result_queue, allow_tracker, end_detector, end_tracker,dataset_metadata_catalog,detector_ready,fps,img_scale_percent,tracker_ready,), nprocs=1, join=False, daemon=False, start_method='spawn')


        detector_p.join()
        tracker_p.join()
        system_p.join()

        #system(videopath, frameQueue, detections_queue, result_queue, allow_tracker, end_detector, end_tracker )


if __name__=="__main__":

    mp.set_start_method("spawn")
    register_coco_instances("zerowaste_test", {}, "/content/drive/MyDrive/inzynierka/datasets/zerowaste-f-final/splits_final_deblurred/test/labels.json", "/content/drive/MyDrive/inzynierka/datasets/zerowaste-f-final/splits_final_deblurred/test/data")
    zerowaste_test_metadata = MetadataCatalog.get("zerowaste_test")
    dataset_dicts_test = DatasetCatalog.get("zerowaste_test")


    cfg = get_cfg()
    cfg.merge_from_file(
        "/content/drive/MyDrive/inzynierka/project/config/detectron2_custom_congif_zerowaste_maskrcnn_R_101_FPN_3x.yaml"
    )
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.65   # set the testing threshold for this model
    predictor = DefaultPredictor(cfg)

    
    detect_and_track_zerowaste(predictor=predictor, dataset_metadata_catalog=zerowaste_test_metadata)

Overwriting /content/project/b.py


In [ ]:
!python b.py

[01/23 01:00:50 d2.data.datasets.coco]: Loaded 929 images in COCO format from /content/drive/MyDrive/inzynierka/datasets/zerowaste-f-final/splits_final_deblurred/test/labels.json
[01/23 01:00:54 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /content/drive/MyDrive/inzynierka/project/detection/maskrcnn/output_zerowasteaug/model_final.pth ...
detector: <Process name='Process-2' parent=8010 started>
detector dummy frame: [1080, 1920, 3]
tracker: <Process name='Process-3' parent=8010 started>
TRACKER dummy frame: [1080, 1920, 3]
/usr/local/lib/python3.8/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
dummy pred time:  4.1815268993377686
predicted dummy
system: <Process name='Process-4' parent=8010 started>
sampling_time: 0.04
TRA

In [ ]:
!python b.py

[01/21 14:48:19 d2.data.datasets.coco]: Loaded 929 images in COCO format from /content/drive/MyDrive/inzynierka/datasets/zerowaste-f-final/splits_final_deblurred/test/labels.json
[01/21 14:48:23 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /content/drive/MyDrive/inzynierka/project/detection/maskrcnn/output_zerowasteaug/model_final.pth ...
detector: <Process name='Process-2' parent=42247 started>
detector dummy frame: [1080, 1920, 3]
tracker: <Process name='Process-3' parent=42247 started>
TRACKER dummy frame: [1080, 1920, 3]
/usr/local/lib/python3.8/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
dummy pred time:  4.283942222595215
predicted dummy
system: <Process name='Process-4' parent=42247 started>
sampling_time: 0.0833

In [ ]:
from deep_sort_realtime.deepsort_tracker import DeepSort
from deep_sort_realtime.deep_sort import kalman_filter

from utils.utils import *
from utils.FrameQueue import FrameQueue, MyManager


def convert_detections_for_deepsort(detections):
    result = []
    for detection in detections:
        x1, y1, x2, y2 = detection[:4]
        x1, x2, w, h = x1y1x2y2_to_x1y1wh(x1.cpu().detach().numpy(), y1.cpu().detach().numpy(), x2.cpu().detach().numpy(), y2.cpu().detach().numpy())
        result.append(([x1, x2, w, h], detection[4].cpu().detach().numpy(), detection[5].cpu().detach().numpy()))
    return result

img_scale_percent=50
fps = 12
    
mot_tracker = DeepSort(max_age=3, max_cosine_distance=0.25, n_init=1, nms_max_overlap=1, img_scale_percent=img_scale_percent)
mot_tracker.tracker.max_iou_distance=0.7
sampling_time = 1/fps
dummy_detections =torch.tensor([[258.7459, 304.2128, 342.2893, 343.4235,   0.8144,   1.0000],
                                [405.6260, 137.3414, 519.5634, 293.4001,   0.7609,   3.0000],
                                [546.6186,  79.7919, 657.5923, 165.6807,   0.7551,   1.0000],
                                [436.6346,  74.1342, 547.4369, 163.3923,   0.7205,   1.0000]])
print(dummy_detections.device)
dets = convert_detections_for_deepsort(dummy_detections)
width = int(1920 * img_scale_percent / 100)
height = int(1080 * img_scale_percent / 100)

print('TRACKER dummy frame:', [height,width,3])
dummy_frame = np.zeros([height,width,3],dtype=np.uint8)
start_dummy = time.time()
mot_tracker.update_tracks(dets, frame=dummy_frame)
print('TRACKER dummy frame time: ', time.time() - start_dummy)

cpu
TRACKER dummy frame: [540, 960, 3]
TRACKER dummy frame time:  0.013973712921142578


# KCF

In [ ]:
%%writefile /content/project/c.py
import torch, detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
# import some common libraries
import numpy as np
import cv2
# from google.colab.patches import cv2_imshow
# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.video_visualizer import VideoVisualizer
from detectron2.utils.visualizer import ColorMode, Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.autograd import Variable

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

import os, sys, time, datetime, random, json
from time import sleep

from multiprocessing.managers import BaseManager
import multiprocessing.queues as mpq
import multiprocessing as mp
import queue

from detectron2.config import get_cfg
import os
from detectron2.data.datasets import register_coco_instances

from utils.utils import *
from utils.FrameQueue import FrameQueue, MyManager

# KCF
from tracking.kcf.KCFMultiTracker import *






def detector(frameQueue, detections_queue, allow_tracker,end_detector, predictor,detector_ready, end_tracker,img_scale_percent):
    print('detector:', mp.current_process())

    #predictor(np.zeros([1080,1920,3],dtype=np.uint8))
    width = int(1920 * img_scale_percent / 100)
    height = int(1080 * img_scale_percent / 100)
    print('detector dummy frame:', [height,width,3])

    dummy_frame = np.zeros([height,width,3],dtype=np.uint8)
    # predictor(np.zeros([height,width,3],dtype=np.uint8))
    start_dummy = time.time()
    detections_output, model_output = detect_image_maskrcnn(dummy_frame, predictor)
    print('dummy pred time: ', time.time() - start_dummy)

    print('predicted dummy')
    detector_ready.set()
    
    while True:
        
        if frameQueue.qsize() > 0:
            break

    while True:
        detector_start = time.process_time()
        try:
            frame, frame_num = frameQueue.get_detection_frame()
        except queue.Empty:
            print('empty')
            break

        print(f"Detector: Running detection on frame {frame_num}")
        ###################
        det_start = time.process_time()
        detections_output, model_output = detect_image_maskrcnn(frame, predictor)
        print('det time:', time.process_time()-det_start)
        # DETECTION PROCESS
        ###################
        print(f"Detector: Done frame {frame_num}")
        if allow_tracker.is_set()==False:
            allow_tracker.set()
        detections_queue.put((frame, frame_num, detections_output, model_output))
        detector_stop = time.process_time()
        print('Detector loop time:', detector_stop - detector_start)
    end_detector.set()
    print('end_detector set')

    while not end_tracker.is_set():
        time.sleep(1)
        continue
    print(' === Detector off === ')

def tracker(frameQueue, detections_queue, result_queue,allow_tracker,end_tracker,fps):

    mot_tracker = MultiTracker(max_age=2, iou_threshold=0.3)
    pca_learning_rate = 0.4 

    class_path='config/zerowaste.names'
    classes = load_classes(class_path)

    sampling_time = 1/fps
    print('tracker:', mp.current_process())
    detect_thresh = 0.4
    param_handler = cv2.TrackerKCF_Params()
    setattr(param_handler, "detect_thresh", detect_thresh)
    detection = None
    while True:
        tracker_start = time.time()

        if not allow_tracker.is_set():
            #print('tracker waiting...', frameQueue.qsize())
            #time.sleep(0.5)
            continue
        
        try:
            tracking_frame = frameQueue.get_tracking_frame()
        except queue.Empty:
            #print('empty')
            break

        if detection != None:
            while detection[1] < tracking_frame[1] and detections_queue.qsize()>0:
                detection = detections_queue.get()

        detections_queue.qsize()
        if detections_queue.qsize()>0:
            if detection==None:
                detection = detections_queue.get()
                if detection[1] == tracking_frame[1]:
                    print(f"Tracker: Running tracking WITH detections {detection[1]} on frame {tracking_frame[1]}")
                    ##################
                    tracker_add_tracks_start = time.time()
                    tracker_update_start = time.time()
                    tracking_boxes, obj_ids, pred_class_names = mot_tracker.update(tracking_frame[0], detection[2])
                    
                    tracked_objects = []
                    for box, obj_id, pred_class_name in zip(tracking_boxes, obj_ids, pred_class_names):
                        (x, y, w, h) = [int(v) for v in box]
                        tracked_objects.append((x, y, x+w, y+h, obj_id, pred_class_name))
                    ##################
                    print(f"Tracker: Done frame {tracking_frame[1]}")
                    result_queue.put((tracking_frame, tracked_objects, detection))
                    detection = None
                else:
                    print(f"Tracker: Running tracking WITHOUT detections on frame {tracking_frame[1]}")

                    ##################
                    tracker_update_start = time.time()
                    tracking_boxes, obj_ids, pred_class_names = mot_tracker.update(tracking_frame[0])
                    
                    tracked_objects = []
                    for box, obj_id, pred_class_name in zip(tracking_boxes, obj_ids, pred_class_names):
                        (x, y, w, h) = [int(v) for v in box]
                        tracked_objects.append((x, y, x+w, y+h, obj_id, pred_class_name))
                    ##################
                    print(f"Tracker: Done frame {tracking_frame[1]}")
                    result_queue.put((tracking_frame, tracked_objects, None))

            else: #  detection!=None
                if detection[1] == tracking_frame[1]:
                    print(f"Tracker: Running tracking WITH detections {detection[1]} on frame {tracking_frame[1]}")
                    ##################
                    tracker_update_start = time.time()
                    tracking_boxes, obj_ids, pred_class_names = mot_tracker.update(tracking_frame[0], detection[2] )
                    
                    tracked_objects = []
                    for box, obj_id, pred_class_name in zip(tracking_boxes, obj_ids, pred_class_names):
                        (x, y, w, h) = [int(v) for v in box]
                        tracked_objects.append((x, y, x+w, y+h, obj_id, pred_class_name))
                    ##################
                    print(f"Tracker: Done frame {tracking_frame[1]}")
                    result_queue.put((tracking_frame, tracked_objects, detection))
                    detection = None

                else:
                    print(f"Tracker: Running tracking WITHOUT detections on frame {tracking_frame[1]}")
                    ##################
                    tracker_update_start = time.time()
                    tracking_boxes, obj_ids, pred_class_names = mot_tracker.update(tracking_frame[0])
                    
                    tracked_objects = []
                    for box, obj_id, pred_class_name in zip(tracking_boxes, obj_ids, pred_class_names):
                        (x, y, w, h) = [int(v) for v in box]
                        tracked_objects.append((x, y, x+w, y+h, obj_id, pred_class_name))
                    ##################
                    print(f"Tracker: Done frame {tracking_frame[1]}")
                    result_queue.put((tracking_frame, tracked_objects, None))
        else: # detections_queue.qsize() == 0
            if detection != None:
                if detection[1] == tracking_frame[1]:
                    print(f"Tracker: Running tracking WITH detections {detection[1]} on frame {tracking_frame[1]}")
                    ##################
                    tracker_update_start = time.time()
                    tracking_boxes, obj_ids, pred_class_names = mot_tracker.update(tracking_frame[0], detection[2])
                    tracked_objects = []
                    for box, obj_id, pred_class_name in zip(tracking_boxes, obj_ids, pred_class_names):
                        (x, y, w, h) = [int(v) for v in box]
                        tracked_objects.append((x, y, x+w, y+h, obj_id, pred_class_name))
                    ##################
                    print(f"Tracker: Done frame {tracking_frame[1]}")
                    result_queue.put((tracking_frame, tracked_objects, detection))
                    # result_queue.task_done()
                    detection = None
                else:
                    print(f"Tracker: Running tracking WITHOUT detections on frame {tracking_frame[1]}")
                    ##################
                    tracker_update_start = time.time()
                    tracking_boxes, obj_ids, pred_class_names = mot_tracker.update(tracking_frame[0])
                    tracked_objects = []
                    for box, obj_id, pred_class_name in zip(tracking_boxes, obj_ids, pred_class_names):
                        (x, y, w, h) = [int(v) for v in box]
                        tracked_objects.append((x, y, x+w, y+h, obj_id, pred_class_name))
                    ##################
                    print(f"Tracker: Done frame {tracking_frame[1]}")
                    result_queue.put((tracking_frame, tracked_objects, None))
            else:
                print(f"Tracker: Running tracking WITHOUT detections on frame {tracking_frame[1]}")
                ##################
                tracker_update_start = time.time()
                tracking_boxes, obj_ids, pred_class_names = mot_tracker.update(tracking_frame[0])
                tracked_objects = []
                for box, obj_id, pred_class_name in zip(tracking_boxes, obj_ids, pred_class_names):
                    (x, y, w, h) = [int(v) for v in box]
                    tracked_objects.append((x, y, x+w, y+h, obj_id, pred_class_name))
                ##################
                print(f"Tracker: Done frame {tracking_frame[1]}")
                result_queue.put((tracking_frame, tracked_objects, None))

        diff = time.time() - tracker_start
        while  diff < sampling_time and frameQueue.qsize()<2:
            diff = time.time() - tracker_start
        print('tracker time:', diff)


    # cleaning detections
    end_tracker.set()
    print('end_tracker set')
    while detections_queue.qsize()>0:
        detections_queue.get()

    while result_queue.qsize()>0:
        time.sleep(1)
        continue

    print(' === Tracker off === ')



def system(videopath, frameQueue, detections_queue, result_queue, allow_tracker, end_detector, end_tracker,dataset_metadata_catalog, detector_ready,fps,img_scale_percent):

    print('system:', mp.current_process())
    ########### Video ###################
    video = cv2.VideoCapture(videopath)
    if (video.isOpened() == False): 
        print("Error reading video file")
    #fps = video.get(cv2.CAP_PROP_FPS)
    sampling_time = 1/fps
    print('sampling_time:',sampling_time)
    frame_width = int(video.get(3))
    frame_height = int(video.get(4))
    size = (frame_width, frame_height)

    #img_scale_percent = 25 # percent of original size
    width = int(frame_width * img_scale_percent / 100)
    height = int(frame_height * img_scale_percent / 100)
    dim = (width, height)
    ###################################
    result = cv2.VideoWriter(f"testing_kcf-{img_scale_percent}-{fps}.avi", 
                            cv2.VideoWriter_fourcc(*'MJPG'),
                            fps, dim)
    ###################################
    class_path='config/zerowaste.names'
    classes = load_classes(class_path)
    counter = 1
    system_start = time.time()
    while(True):
        if not detector_ready.is_set():
            continue

        start = time.time()

        ret, frame = video.read()
        if ret == True:
            print('------------------ ' + str(counter) +' ------------------')
            resized_frame = cv2.resize(frame, dim, interpolation = cv2.INTER_AREA)
            frameQueue.add_frame((resized_frame, counter))
            # print('frameQueue:', [element[1] for element in frameQueue.queue])
            frameQueue.print()

            if allow_tracker.is_set() == False:      
                print('--> Add to queue and go to next frame')
                counter += 1
                diff = time.time() - start

                while  diff < sampling_time:
                    diff = time.time() - start
                print(diff) 
                continue

        elif ret == False:
            break
        
        counter += 1

        diff = time.time() - start
        while  diff < sampling_time:
            diff = time.time() - start
        print(diff)
        

    while True:
        if end_detector.is_set() and end_tracker.is_set():
            break
    system_stop = time.time()
    print('System time = ', system_stop-system_start)
    

    v = VideoVisualizer(dataset_metadata_catalog, ColorMode.IMAGE)
    print(' \\\\\\\\\\\ Saving frames to video')
    while result_queue.qsize()>0: # and (detections_queue.empty() and tracking_queue.empty())
        (tracking_frame, tracking_frame_num), tracked_objects, detection = result_queue.get()
        if detection is not None:
            detection_frame, detection_frame_num, detections_output, model_output = detection
            visualizer = v.draw_instance_predictions(detection_frame, model_output["instances"])
            img_pred = visualizer.get_image()
            cv2.putText(img_pred,  "DETECTION", (5, 40), cv2.FONT_HERSHEY_DUPLEX, 0.4, (0,255,0), 1)
            tracking_frame = img_pred
        for x1, y1, x2, y2, obj_id, cls_pred in tracked_objects:
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            cls = cls_pred
            cv2.rectangle(tracking_frame, (x1, y1), (x2, y2), (0, 0, 255), 1)
            cv2.putText(tracking_frame, cls + "-" + str(int(obj_id)), (x1, y1 + 10), cv2.FONT_HERSHEY_DUPLEX, 0.4, (255,255,255), 1)

        cv2.putText(tracking_frame,  f"FRAME: {tracking_frame_num} | FPS: {fps}", (5, 15), cv2.FONT_HERSHEY_DUPLEX, 0.5, (255,255,255), 1)
        result.write(tracking_frame)
    ########################################

    result.release()
    video.release()
    # Closes all the frames
    cv2.destroyAllWindows()
    print("The video was successfully saved")


def detect_and_track_zerowaste(predictor,dataset_metadata_catalog):

    MyManager.register('FrameQueue', FrameQueue)
    fps = 12
    img_scale_percent = 35

    # create a new manager instance
    with MyManager() as manager:
        # create a shared custom class instance
        frameQueue = manager.FrameQueue()
        detections_queue = mp.Queue()#mp.Queue()
        result_queue = mp.Queue()#mp.Queue()
        videopath = '/content/drive/MyDrive/inzynierka/project/videos/05_frame-10-12.mp4'

        allow_tracker = mp.Event()
        end_detector = mp.Event()
        end_tracker = mp.Event()
        detector_ready = mp.Event()

        tracker_p = mp.Process(target=tracker, args=(frameQueue,detections_queue,result_queue,allow_tracker,end_tracker,fps,))
        tracker_p.start()

        detector_p = mp.Process(target=detector, args=(frameQueue,detections_queue,allow_tracker,end_detector,predictor,detector_ready,end_tracker,img_scale_percent,))
        detector_p.start()
        
        system_p = mp.Process(target=system, args=(videopath, frameQueue, detections_queue, result_queue, allow_tracker, end_detector, end_tracker,dataset_metadata_catalog,detector_ready,fps,img_scale_percent,))
        system_p.start()


        
        detector_p.join()
        tracker_p.join()
        system_p.join()


        #system(videopath, frameQueue, detections_queue, result_queue, allow_tracker, end_detector, end_tracker )



if __name__=="__main__":

    mp.set_start_method("spawn")
    register_coco_instances("zerowaste_test", {}, "/content/drive/MyDrive/inzynierka/datasets/zerowaste-f-final/splits_final_deblurred/test/labels.json", "/content/drive/MyDrive/inzynierka/datasets/zerowaste-f-final/splits_final_deblurred/test/data")
    zerowaste_test_metadata = MetadataCatalog.get("zerowaste_test")
    dataset_dicts_test = DatasetCatalog.get("zerowaste_test")



    cfg = get_cfg()
    cfg.merge_from_file(
        "/content/drive/MyDrive/inzynierka/project/config/detectron2_custom_congif_zerowaste_maskrcnn_R_101_FPN_3x.yaml"
    )
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.65   # set the testing threshold for this model
    predictor = DefaultPredictor(cfg)

    
    detect_and_track_zerowaste(predictor=predictor, dataset_metadata_catalog=zerowaste_test_metadata)

Overwriting /content/project/c.py


In [ ]:
!python c.py

[01/21 14:54:59 d2.data.datasets.coco]: Loaded 929 images in COCO format from /content/drive/MyDrive/inzynierka/datasets/zerowaste-f-final/splits_final_deblurred/test/labels.json
[01/21 14:55:02 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /content/drive/MyDrive/inzynierka/project/detection/maskrcnn/output_zerowasteaug/model_final.pth ...
tracker: <Process name='Process-2' parent=44244 started>
detector: <Process name='Process-3' parent=44244 started>
detector dummy frame: [378, 672, 3]
system: <Process name='Process-4' parent=44244 started>
sampling_time: 0.08333333333333333
/usr/local/lib/python3.8/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
dummy pred time:  4.088770151138306
predicted dummy
------------------ 1 ----

In [ ]:
!python c.py

[01/20 12:22:16 d2.data.datasets.coco]: Loaded 929 images in COCO format from /content/drive/MyDrive/inzynierka/datasets/zerowaste-f-final/splits_final_deblurred/test/labels.json
[01/20 12:22:19 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /content/drive/MyDrive/inzynierka/project/detection/maskrcnn/output_zerowasteaug/model_final.pth ...
tracker: <Process name='Process-2' parent=16689 started>
detector: <Process name='Process-3' parent=16689 started>
system: <Process name='Process-4' parent=16689 started>
sampling_time: 0.125
/usr/local/lib/python3.8/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
predicted dummy
------------------ 1 ------------------
[1]
--> Add to queue and go to next frame
0.20324420928955078
Detector: